I obtained historical closing line data (decimal/European odds format) from Dan Westin (http://www.tennis-data.co.uk/alldata.php). The goal in this workbook is to wrangle the closing line data (just using average across a number of books currently) into a format that it can be accurately merged into each surface-specific core dataframe (Workbook 2; clay and hard separate). 

At the end of this workbook, there is some sorcery to convert the decimal odds to implied win probabilities for each player in a given match. These implied win probabilities (summing to exactly 100% per match) have the vig removed (very important to do, since books bake in their profits to assymetry in the lines).  

NOTE: I cleaned up some inconsistencies in Dan's files manually before importing them here (hence, "...JNR_amended")

In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Read file into a DataFrame and print head. Set missing values to NaN
df_1 = pd.ExcelFile('../data/historical_wagering_data/JNR_amended_files/2012_JNR_amended.xlsx', engine='openpyxl')
df_2012 = df_1.parse('2012')
df_2 = pd.ExcelFile('../data/historical_wagering_data/JNR_amended_files/2013_JNR_amended.xlsx', engine='openpyxl')
df_2013 = df_2.parse('2013')
df_3 = pd.ExcelFile('../data/historical_wagering_data/JNR_amended_files/2014_JNR_amended.xlsx', engine='openpyxl')
df_2014 = df_3.parse('2014')
df_4 = pd.ExcelFile('../data/historical_wagering_data/JNR_amended_files/2015_JNR_amended.xlsx', engine='openpyxl')
df_2015 = df_4.parse('2015')
df_5 = pd.ExcelFile('../data/historical_wagering_data/JNR_amended_files/2016_JNR_amended.xlsx', engine='openpyxl')
df_2016 = df_5.parse('2016')
df_6 = pd.ExcelFile('../data/historical_wagering_data/JNR_amended_files/2017_JNR_amended.xlsx', engine='openpyxl')
df_2017 = df_6.parse('2017')
df_7 = pd.ExcelFile('../data/historical_wagering_data/JNR_amended_files/2018_JNR_amended.xlsx', engine='openpyxl')
df_2018 = df_7.parse('2018')
df_8 = pd.ExcelFile('../data/historical_wagering_data/JNR_amended_files/2019_JNR_amended.xlsx', engine='openpyxl')
df_2019 = df_8.parse('2019')

In [3]:
df_2019.head()

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,...,Lsets,Comment,B365W,B365L,PSW,PSL,MaxW,MaxL,AvgW,AvgL
0,1,Brisbane,Brisbane International,2019-01-01,ATP250,Outdoor,Hard,1st Round,3,Dimitrov G.,...,0.0,Completed,1.36,3.00,1.36,3.37,1.42,3.60,1.35,3.18
1,1,Brisbane,Brisbane International,2019-01-01,ATP250,Outdoor,Hard,1st Round,3,Raonic M.,...,0.0,Completed,1.18,4.50,1.23,4.68,1.27,4.84,1.22,4.26
2,1,Brisbane,Brisbane International,2019-01-01,ATP250,Outdoor,Hard,1st Round,3,Kecmanovic M.,...,0.0,Completed,1.57,2.25,1.67,2.32,1.71,2.40,1.63,2.28
3,1,Brisbane,Brisbane International,2019-01-01,ATP250,Outdoor,Hard,1st Round,3,Millman J.,...,1.0,Completed,1.40,2.75,1.41,3.13,1.45,3.20,1.40,2.95
4,1,Brisbane,Brisbane International,2019-01-01,ATP250,Outdoor,Hard,1st Round,3,Uchiyama Y.,...,0.0,Completed,2.62,1.44,2.73,1.51,3.26,1.53,2.69,1.47


In [4]:
df = pd.concat([df_2019, df_2018, df_2017, df_2016, df_2015, df_2014, df_2013, df_2012], ignore_index=True)
df.head(20) 

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,...,MaxW,MaxL,AvgW,AvgL,EXW,EXL,LBW,LBL,SJW,SJL
0,1,Brisbane,Brisbane International,2019-01-01,ATP250,Outdoor,Hard,1st Round,3,Dimitrov G.,...,1.42,3.60,1.35,3.18,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Brisbane,Brisbane International,2019-01-01,ATP250,Outdoor,Hard,1st Round,3,Raonic M.,...,1.27,4.84,1.22,4.26,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Brisbane,Brisbane International,2019-01-01,ATP250,Outdoor,Hard,1st Round,3,Kecmanovic M.,...,1.71,2.40,1.63,2.28,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Brisbane,Brisbane International,2019-01-01,ATP250,Outdoor,Hard,1st Round,3,Millman J.,...,1.45,3.20,1.40,2.95,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Brisbane,Brisbane International,2019-01-01,ATP250,Outdoor,Hard,1st Round,3,Uchiyama Y.,...,3.26,1.53,2.69,1.47,NaN,NaN,NaN,NaN,NaN,NaN
5,1,Brisbane,Brisbane International,2019-01-01,ATP250,Outdoor,Hard,1st Round,3,Kudla D.,...,2.85,1.55,2.70,1.47,NaN,NaN,NaN,NaN,NaN,NaN
6,1,Brisbane,Brisbane International,2019-01-01,ATP250,Outdoor,Hard,1st Round,3,Chardy J.,...,2.26,1.74,2.19,1.68,NaN,NaN,NaN,NaN,NaN,NaN
7,1,Brisbane,Brisbane International,2019-01-01,ATP250,Outdoor,Hard,1st Round,3,Murray A.,...,1.39,3.60,1.34,3.26,NaN,NaN,NaN,NaN,NaN,NaN
8,1,Brisbane,Brisbane International,2019-01-01,ATP250,Outdoor,Hard,1st Round,3,Kyrgios N.,...,1.50,3.16,1.44,2.80,NaN,NaN,NaN,NaN,NaN,NaN
9,1,Brisbane,Brisbane International,2019-01-01,ATP250,Outdoor,Hard,1st Round,3,Tsonga J.W.,...,2.42,1.71,2.27,1.64,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Strip the df down to only what we will need to set up for the merge with core dataframe. It's a little kludgy (we have no player id numbers from Dan's dataframe and the player name convention is different from the core dataframe too unfortunately), but the player rankings at time of match (a unique, albeit time-dependent, identifier) combined with a tour week identifier that we will generate will be sufficient to get the wagering info in place in the core dataframe. 
# A considerable amount of cross-checking to the core dataframe (ATP-validated data) and amending rankings at a given time has been manually acheived in these 'JNR-amended' data files. Other small items
# in Dan's data sets have also been amended manually (based on feedback from early rounds of merge attempts). 

df2 = df[['Location','Tournament','Date','Surface','Round','Winner','Loser', 'WRank', 'LRank', 'AvgW', 'AvgL','Comment']]

In [6]:
# Renaming remaining columns per core dataframe
df2.rename(columns = {'Location':'t_loc','Tournament':'t_name','Surface':'t_surf','Round':'t_round','Winner':'w_name','Loser':'l_name','WRank':'w_rank','LRank':'l_rank','AvgW':'AvgW_BL','AvgL':'AvgL_BL'}, inplace=True)

In [7]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20974 entries, 0 to 20973
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   t_loc    20974 non-null  object        
 1   t_name   20974 non-null  object        
 2   Date     20974 non-null  datetime64[ns]
 3   t_surf   20974 non-null  object        
 4   t_round  20974 non-null  object        
 5   w_name   20974 non-null  object        
 6   l_name   20974 non-null  object        
 7   w_rank   20964 non-null  float64       
 8   l_rank   20924 non-null  float64       
 9   AvgW_BL  20960 non-null  float64       
 10  AvgL_BL  20960 non-null  float64       
 11  Comment  20974 non-null  object        
dtypes: datetime64[ns](1), float64(4), object(7)
memory usage: 1.9+ MB


In [8]:
# Filter out grass court matches and matches that didn't happen
df2 = df2[~df2['t_surf'].str.contains("Grass")]
df2 = df2[~df2['Comment'].str.contains("Walkover")]

In [9]:
# Convert datetime in Date to string to make useful for renaming indexing below
df2['Date'] = df2['Date'].dt.strftime('%Y-%m-%d') #we want datetime as a string to index into here

In [10]:
# Rename some tournments to avoid ambiguity (since we are using t_loc, not Tourny Name)
df2.loc[(df2["t_name"] == "French Open"), "t_loc"] = "Roland Garros" #avoid ambiguity with Paris Indoor
df2.loc[(df2["t_name"] == "US Open"), "t_loc"] = "Flushing Meadows" #avoid ambiguity with New York ATP event
df2.loc[(df2["t_loc"] == "Dubai "), "t_loc"] = "Dubai" #extra space bug in Dan's files for Dubai
df2.loc[(df2["t_loc"] == "Shenzhen "), "t_loc"] = "Shenzhen" #same for Shenzen
df2.loc[(df2["t_loc"] == "Estoril "), "t_loc"] = "Estoril" #same for Estoril

In [11]:
# Create proper 'tour_week' for each tourney in the sample. This will be a key column to get an accurate merge with the core dataframe
df2.loc[(df2["Date"].str.contains("2019")) & ((df2["t_loc"] == "Brisbane")|(df2["t_loc"] == "Doha")|(df2["t_loc"] == "Pune")), "tour_wk"] = "2019_01"
df2.loc[(df2["Date"].str.contains("2019")) & ((df2["t_loc"] == "Auckland")|(df2["t_loc"] == "Sydney")), "tour_wk"] = "2019_02"       
df2.loc[(df2["Date"].str.contains("2019")) & ((df2["t_loc"] == "Melbourne")), "tour_wk"] = "2019_03"       
df2.loc[(df2["Date"].str.contains("2019")) & ((df2["t_loc"] == "Cordoba")|(df2["t_loc"] == "Montpellier")|(df2["t_loc"] == "Sofia")), "tour_wk"] = "2019_04"     
df2.loc[(df2["Date"].str.contains("2019")) & ((df2["t_loc"] == "Buenos Aires")|(df2["t_loc"] == "Rotterdam")|(df2["t_loc"] == "New York")), "tour_wk"] = "2019_05" 
df2.loc[(df2["Date"].str.contains("2019")) & ((df2["t_loc"] == "Delray Beach")|(df2["t_loc"] == "Marseille")|(df2["t_loc"] == "Rio de Janeiro")), "tour_wk"] = "2019_06"
df2.loc[(df2["Date"].str.contains("2019")) & ((df2["t_loc"] == "Acapulco")|(df2["t_loc"] == "Dubai")|(df2["t_loc"] == "Sao Paulo")), "tour_wk"] = "2019_07"
df2.loc[(df2["Date"].str.contains("2019")) & ((df2["t_loc"] == "Indian Wells")), "tour_wk"] = "2019_08"  
df2.loc[(df2["Date"].str.contains("2019")) & ((df2["t_loc"] == "Miami")), "tour_wk"] = "2019_09"
df2.loc[(df2["Date"].str.contains("2019")) & ((df2["t_loc"] == "Houston")|(df2["t_loc"] == "Marrakech")), "tour_wk"] = "2019_10"  
df2.loc[(df2["Date"].str.contains("2019")) & ((df2["t_loc"] == "Monte Carlo")), "tour_wk"] = "2019_11"
df2.loc[(df2["Date"].str.contains("2019")) & ((df2["t_loc"] == "Barcelona")|(df2["t_loc"] == "Budapest")), "tour_wk"] = "2019_12"  
df2.loc[(df2["Date"].str.contains("2019")) & ((df2["t_loc"] == "Estoril")|(df2["t_loc"] == "Munich")), "tour_wk"] = "2019_13"
df2.loc[(df2["Date"].str.contains("2019")) & ((df2["t_loc"] == "Madrid")), "tour_wk"] = "2019_14"
df2.loc[(df2["Date"].str.contains("2019")) & ((df2["t_loc"] == "Rome")), "tour_wk"] = "2019_15"
df2.loc[(df2["Date"].str.contains("2019")) & ((df2["t_loc"] == "Geneva")|(df2["t_loc"] == "Lyon")), "tour_wk"] = "2019_16"
df2.loc[(df2["Date"].str.contains("2019")) & ((df2["t_loc"] == "Roland Garros")), "tour_wk"] = "2019_17"
df2.loc[(df2["Date"].str.contains("2019")) & ((df2["t_loc"] == "Bastad")|(df2["t_loc"] == "Umag")), "tour_wk"] = "2019_18"
df2.loc[(df2["Date"].str.contains("2019")) & (((df2["t_loc"] == "Atlanta")|(df2["t_loc"] == "Gstaad")|(df2["t_loc"] == "Hamburg"))), "tour_wk"] = "2019_19"
df2.loc[(df2["Date"].str.contains("2019")) & (((df2["t_loc"] == "Kitzbuhel")|(df2["t_loc"] == "Los Cabos")|(df2["t_loc"] == "Washington"))), "tour_wk"] = "2019_20"
df2.loc[(df2["Date"].str.contains("2019")) & ((df2["t_loc"] == "Montreal")), "tour_wk"] = "2019_21"
df2.loc[(df2["Date"].str.contains("2019")) & ((df2["t_loc"] == "Cincinnati")), "tour_wk"] = "2019_22"
df2.loc[(df2["Date"].str.contains("2019")) & ((df2["t_loc"] == "Winston-Salem")), "tour_wk"] = "2019_23"
df2.loc[(df2["Date"].str.contains("2019")) & ((df2["t_loc"] == "Flushing Meadows")), "tour_wk"] = "2019_24"
df2.loc[(df2["Date"].str.contains("2019")) & ((df2["t_loc"] == "Metz")|(df2["t_loc"] == "St. Petersburg")), "tour_wk"] = "2019_25"  
df2.loc[(df2["Date"].str.contains("2019")) & ((df2["t_loc"] == "Chengdu")|(df2["t_loc"] == "Zhuhai")), "tour_wk"] = "2019_26"  
df2.loc[(df2["Date"].str.contains("2019")) & ((df2["t_loc"] == "Beijing")|(df2["t_loc"] == "Tokyo")), "tour_wk"] = "2019_27"  
df2.loc[(df2["Date"].str.contains("2019")) & ((df2["t_loc"] == "Shanghai")), "tour_wk"] = "2019_28"
df2.loc[(df2["Date"].str.contains("2019")) & (((df2["t_loc"] == "Antwerp")|(df2["t_loc"] == "Moscow")|(df2["t_loc"] == "Stockholm"))), "tour_wk"] = "2019_29"
df2.loc[(df2["Date"].str.contains("2019")) & ((df2["t_loc"] == "Basel")|(df2["t_loc"] == "Vienna")), "tour_wk"] = "2019_30"
df2.loc[(df2["Date"].str.contains("2019")) & ((df2["t_loc"] == "Paris")), "tour_wk"] = "2019_31"
df2.loc[(df2["Date"].str.contains("2019")) & ((df2["t_loc"] == "London")), "tour_wk"] = "2019_32"
df2.loc[(df2["Date"].str.contains("2019")) & ((df2["t_loc"] == "NextGen Finals")), "tour_wk"] = "2019_33"

df2.loc[(df2["Date"].str.contains("2018")) & (((df2["t_loc"] == "Brisbane")|(df2["t_loc"] == "Doha")|(df2["t_loc"] == "Pune"))), "tour_wk"] = "2018_01"
df2.loc[(df2["Date"].str.contains("2018")) & ((df2["t_loc"] == "Auckland")|(df2["t_loc"] == "Sydney")), "tour_wk"] = "2018_02"       
df2.loc[(df2["Date"].str.contains("2018")) & ((df2["t_loc"] == "Melbourne")), "tour_wk"] = "2018_03"       
df2.loc[(df2["Date"].str.contains("2018")) & ((df2["t_loc"] == "Quito")|(df2["t_loc"] == "Montpellier")|(df2["t_loc"] == "Sofia")), "tour_wk"] = "2018_04"     
df2.loc[(df2["Date"].str.contains("2018")) & ((df2["t_loc"] == "Buenos Aires")|(df2["t_loc"] == "Rotterdam")|(df2["t_loc"] == "New York")), "tour_wk"] = "2018_05" 
df2.loc[(df2["Date"].str.contains("2018")) & ((df2["t_loc"] == "Delray Beach")|(df2["t_loc"] == "Marseille")|(df2["t_loc"] == "Rio de Janeiro")), "tour_wk"] = "2018_06"
df2.loc[(df2["Date"].str.contains("2018")) & ((df2["t_loc"] == "Acapulco")|(df2["t_loc"] == "Dubai")|(df2["t_loc"] == "Sao Paulo")), "tour_wk"] = "2018_07"
df2.loc[(df2["Date"].str.contains("2018")) & ((df2["t_loc"] == "Indian Wells")), "tour_wk"] = "2018_08"  
df2.loc[(df2["Date"].str.contains("2018")) & ((df2["t_loc"] == "Miami")), "tour_wk"] = "2018_09"
df2.loc[(df2["Date"].str.contains("2018")) & ((df2["t_loc"] == "Houston")|(df2["t_loc"] == "Marrakech")), "tour_wk"] = "2018_10"  
df2.loc[(df2["Date"].str.contains("2018")) & ((df2["t_loc"] == "Monte Carlo")), "tour_wk"] = "2018_11"
df2.loc[(df2["Date"].str.contains("2018")) & ((df2["t_loc"] == "Barcelona")|(df2["t_loc"] == "Budapest")), "tour_wk"] = "2018_12"  
df2.loc[(df2["Date"].str.contains("2018")) & ((df2["t_loc"] == "Estoril")|(df2["t_loc"] == "Munich") |(df2["t_loc"] == "Istanbul")), "tour_wk"] = "2018_13"
df2.loc[(df2["Date"].str.contains("2018")) & ((df2["t_loc"] == "Madrid")), "tour_wk"] = "2018_14"
df2.loc[(df2["Date"].str.contains("2018")) & ((df2["t_loc"] == "Rome")), "tour_wk"] = "2018_15"
df2.loc[(df2["Date"].str.contains("2018")) & ((df2["t_loc"] == "Geneva")|(df2["t_loc"] == "Lyon")), "tour_wk"] = "2018_16"
df2.loc[(df2["Date"].str.contains("2018")) & ((df2["t_loc"] == "Roland Garros")), "tour_wk"] = "2018_17"
df2.loc[(df2["Date"].str.contains("2018")) & ((df2["t_loc"] == "Bastad")|(df2["t_loc"] == "Umag")), "tour_wk"] = "2018_18"
df2.loc[(df2["Date"].str.contains("2018")) & (((df2["t_loc"] == "Atlanta")|(df2["t_loc"] == "Gstaad")|(df2["t_loc"] == "Hamburg"))), "tour_wk"] = "2018_19"
df2.loc[(df2["Date"].str.contains("2018")) & (((df2["t_loc"] == "Kitzbuhel")|(df2["t_loc"] == "Los Cabos")|(df2["t_loc"] == "Washington"))), "tour_wk"] = "2018_20"
df2.loc[(df2["Date"].str.contains("2018")) & ((df2["t_loc"] == "Toronto")), "tour_wk"] = "2018_21"
df2.loc[(df2["Date"].str.contains("2018")) & ((df2["t_loc"] == "Cincinnati")), "tour_wk"] = "2018_22"
df2.loc[(df2["Date"].str.contains("2018")) & ((df2["t_loc"] == "Winston-Salem")), "tour_wk"] = "2018_23"
df2.loc[(df2["Date"].str.contains("2018")) & ((df2["t_loc"] == "Flushing Meadows")), "tour_wk"] = "2018_24"
df2.loc[(df2["Date"].str.contains("2018")) & ((df2["t_loc"] == "Metz")|(df2["t_loc"] == "St. Petersburg")), "tour_wk"] = "2018_25"  
df2.loc[(df2["Date"].str.contains("2018")) & ((df2["t_loc"] == "Chengdu")|(df2["t_loc"] == "Shenzhen")), "tour_wk"] = "2018_26"  
df2.loc[(df2["Date"].str.contains("2018")) & ((df2["t_loc"] == "Beijing")|(df2["t_loc"] == "Tokyo")), "tour_wk"] = "2018_27"  
df2.loc[(df2["Date"].str.contains("2018")) & ((df2["t_loc"] == "Shanghai")), "tour_wk"] = "2018_28"
df2.loc[(df2["Date"].str.contains("2018")) & (((df2["t_loc"] == "Antwerp")|(df2["t_loc"] == "Moscow")|(df2["t_loc"] == "Stockholm"))), "tour_wk"] = "2018_29"
df2.loc[(df2["Date"].str.contains("2018")) & ((df2["t_loc"] == "Basel")|(df2["t_loc"] == "Vienna")), "tour_wk"] = "2018_30"
df2.loc[(df2["Date"].str.contains("2018")) & ((df2["t_loc"] == "Paris")), "tour_wk"] = "2018_31"
df2.loc[(df2["Date"].str.contains("2018")) & ((df2["t_loc"] == "NextGen Finals")), "tour_wk"] = "2018_32"
df2.loc[(df2["Date"].str.contains("2018")) & ((df2["t_loc"] == "London")), "tour_wk"] = "2018_33"

df2.loc[(df2["Date"].str.contains("2017")) & (((df2["t_loc"] == "Brisbane")|(df2["t_loc"] == "Doha")|(df2["t_loc"] == "Chennai"))), "tour_wk"] = "2017_01"
df2.loc[(df2["Date"].str.contains("2017")) & ((df2["t_loc"] == "Auckland")|(df2["t_loc"] == "Sydney")), "tour_wk"] = "2017_02"       
df2.loc[(df2["Date"].str.contains("2017")) & ((df2["t_loc"] == "Melbourne")), "tour_wk"] = "2017_03"       
df2.loc[(df2["Date"].str.contains("2017")) & ((df2["t_loc"] == "Quito")|(df2["t_loc"] == "Montpellier")|(df2["t_loc"] == "Sofia")), "tour_wk"] = "2017_04"     
df2.loc[(df2["Date"].str.contains("2017")) & ((df2["t_loc"] == "Buenos Aires")|(df2["t_loc"] == "Rotterdam")|(df2["t_loc"] == "Memphis")), "tour_wk"] = "2017_05" 
df2.loc[(df2["Date"].str.contains("2017")) & ((df2["t_loc"] == "Delray Beach")|(df2["t_loc"] == "Marseille")|(df2["t_loc"] == "Rio de Janeiro")), "tour_wk"] = "2017_06"
df2.loc[(df2["Date"].str.contains("2017")) & ((df2["t_loc"] == "Acapulco")|(df2["t_loc"] == "Dubai")|(df2["t_loc"] == "Sao Paulo")), "tour_wk"] = "2017_07"
df2.loc[(df2["Date"].str.contains("2017")) & ((df2["t_loc"] == "Indian Wells")), "tour_wk"] = "2017_08"  
df2.loc[(df2["Date"].str.contains("2017")) & ((df2["t_loc"] == "Miami")), "tour_wk"] = "2017_09"
df2.loc[(df2["Date"].str.contains("2017")) & ((df2["t_loc"] == "Houston")|(df2["t_loc"] == "Marrakech")), "tour_wk"] = "2017_10"  
df2.loc[(df2["Date"].str.contains("2017")) & ((df2["t_loc"] == "Monte Carlo")), "tour_wk"] = "2017_11"
df2.loc[(df2["Date"].str.contains("2017")) & ((df2["t_loc"] == "Barcelona")|(df2["t_loc"] == "Budapest")), "tour_wk"] = "2017_12"  
df2.loc[(df2["Date"].str.contains("2017")) & ((df2["t_loc"] == "Estoril")|(df2["t_loc"] == "Munich") |(df2["t_loc"] == "Istanbul")), "tour_wk"] = "2017_13"
df2.loc[(df2["Date"].str.contains("2017")) & ((df2["t_loc"] == "Madrid")), "tour_wk"] = "2017_14"
df2.loc[(df2["Date"].str.contains("2017")) & ((df2["t_loc"] == "Rome")), "tour_wk"] = "2017_15"
df2.loc[(df2["Date"].str.contains("2017")) & ((df2["t_loc"] == "Geneva")|(df2["t_loc"] == "Lyon")), "tour_wk"] = "2017_16"
df2.loc[(df2["Date"].str.contains("2017")) & ((df2["t_loc"] == "Roland Garros")), "tour_wk"] = "2017_17"
df2.loc[(df2["Date"].str.contains("2017")) & ((df2["t_loc"] == "Bastad")|(df2["t_loc"] == "Umag")), "tour_wk"] = "2017_18"
df2.loc[(df2["Date"].str.contains("2017")) & (((df2["t_loc"] == "Atlanta")|(df2["t_loc"] == "Gstaad")|(df2["t_loc"] == "Hamburg"))), "tour_wk"] = "2017_19"
df2.loc[(df2["Date"].str.contains("2017")) & (((df2["t_loc"] == "Kitzbuhel")|(df2["t_loc"] == "Los Cabos")|(df2["t_loc"] == "Washington"))), "tour_wk"] = "2017_20"
df2.loc[(df2["Date"].str.contains("2017")) & ((df2["t_loc"] == "Montreal")), "tour_wk"] = "2017_21"
df2.loc[(df2["Date"].str.contains("2017")) & ((df2["t_loc"] == "Cincinnati")), "tour_wk"] = "2017_22"
df2.loc[(df2["Date"].str.contains("2017")) & ((df2["t_loc"] == "Winston-Salem")), "tour_wk"] = "2017_23"
df2.loc[(df2["Date"].str.contains("2017")) & ((df2["t_loc"] == "Flushing Meadows")), "tour_wk"] = "2017_24"
df2.loc[(df2["Date"].str.contains("2017")) & ((df2["t_loc"] == "Metz")|(df2["t_loc"] == "St. Petersburg")), "tour_wk"] = "2017_25"  
df2.loc[(df2["Date"].str.contains("2017")) & ((df2["t_loc"] == "Chengdu")|(df2["t_loc"] == "Shenzhen")), "tour_wk"] = "2017_26"  
df2.loc[(df2["Date"].str.contains("2017")) & ((df2["t_loc"] == "Beijing")|(df2["t_loc"] == "Tokyo")), "tour_wk"] = "2017_27"  
df2.loc[(df2["Date"].str.contains("2017")) & ((df2["t_loc"] == "Shanghai")), "tour_wk"] = "2017_28"
df2.loc[(df2["Date"].str.contains("2017")) & (((df2["t_loc"] == "Antwerp")|(df2["t_loc"] == "Moscow")|(df2["t_loc"] == "Stockholm"))), "tour_wk"] = "2017_29"
df2.loc[(df2["Date"].str.contains("2017")) & ((df2["t_loc"] == "Basel")|(df2["t_loc"] == "Vienna")), "tour_wk"] = "2017_30"
df2.loc[(df2["Date"].str.contains("2017")) & ((df2["t_loc"] == "Paris")), "tour_wk"] = "2017_31"
df2.loc[(df2["Date"].str.contains("2017")) & ((df2["t_loc"] == "London")), "tour_wk"] = "2017_32"
df2.loc[(df2["Date"].str.contains("2017")) & ((df2["t_loc"] == "NextGen Finals")), "tour_wk"] = "2017_33"

df2.loc[(df2["Date"].str.contains("2016")) & (((df2["t_loc"] == "Brisbane")|(df2["t_loc"] == "Doha")|(df2["t_loc"] == "Chennai"))), "tour_wk"] = "2016_01"
df2.loc[(df2["Date"].str.contains("2016")) & ((df2["t_loc"] == "Auckland")|(df2["t_loc"] == "Sydney")), "tour_wk"] = "2016_02"       
df2.loc[(df2["Date"].str.contains("2016")) & ((df2["t_loc"] == "Melbourne")), "tour_wk"] = "2016_03"       
df2.loc[(df2["Date"].str.contains("2016")) & ((df2["t_loc"] == "Quito")|(df2["t_loc"] == "Montpellier")|(df2["t_loc"] == "Sofia")), "tour_wk"] = "2016_04"     
df2.loc[(df2["Date"].str.contains("2016")) & ((df2["t_loc"] == "Buenos Aires")|(df2["t_loc"] == "Rotterdam")|(df2["t_loc"] == "Memphis")), "tour_wk"] = "2016_05" 
df2.loc[(df2["Date"].str.contains("2016")) & ((df2["t_loc"] == "Delray Beach")|(df2["t_loc"] == "Marseille")|(df2["t_loc"] == "Rio de Janeiro")), "tour_wk"] = "2016_06"
df2.loc[(df2["Date"].str.contains("2016")) & ((df2["t_loc"] == "Acapulco")|(df2["t_loc"] == "Dubai")|(df2["t_loc"] == "Sao Paulo")), "tour_wk"] = "2016_07"
df2.loc[(df2["Date"].str.contains("2016")) & ((df2["t_loc"] == "Indian Wells")), "tour_wk"] = "2016_08"  
df2.loc[(df2["Date"].str.contains("2016")) & ((df2["t_loc"] == "Miami")), "tour_wk"] = "2016_09"
df2.loc[(df2["Date"].str.contains("2016")) & ((df2["t_loc"] == "Houston")|(df2["t_loc"] == "Marrakech")), "tour_wk"] = "2016_10"  
df2.loc[(df2["Date"].str.contains("2016")) & ((df2["t_loc"] == "Monte Carlo")), "tour_wk"] = "2016_11"
df2.loc[(df2["Date"].str.contains("2016")) & ((df2["t_loc"] == "Barcelona")|(df2["t_loc"] == "Bucharest")), "tour_wk"] = "2016_12"  
df2.loc[(df2["Date"].str.contains("2016")) & ((df2["t_loc"] == "Estoril")|(df2["t_loc"] == "Munich") |(df2["t_loc"] == "Istanbul")), "tour_wk"] = "2016_13"
df2.loc[(df2["Date"].str.contains("2016")) & ((df2["t_loc"] == "Madrid")), "tour_wk"] = "2016_14"
df2.loc[(df2["Date"].str.contains("2016")) & ((df2["t_loc"] == "Rome")), "tour_wk"] = "2016_15"
df2.loc[(df2["Date"].str.contains("2016")) & ((df2["t_loc"] == "Geneva")|(df2["t_loc"] == "Nice")), "tour_wk"] = "2016_16"
df2.loc[(df2["Date"].str.contains("2016")) & ((df2["t_loc"] == "Roland Garros")), "tour_wk"] = "2016_17"
df2.loc[(df2["Date"].str.contains("2016")) & ((df2["t_loc"] == "Bastad") |(df2["t_loc"] == "Hamburg")), "tour_wk"] = "2016_18"
df2.loc[(df2["Date"].str.contains("2016")) & ((df2["t_loc"] == "Umag")|(df2["t_loc"] == "Kitzbuhel")| (df2["t_loc"] == "Gstaad")|(df2["t_loc"] == "Washington")), "tour_wk"] = "2016_19"
df2.loc[(df2["Date"].str.contains("2016")) & ((df2["t_loc"] == "Toronto")), "tour_wk"] = "2016_20"
df2.loc[(df2["Date"].str.contains("2016")) & ((df2["t_loc"] == "Atlanta")), "tour_wk"] = "2016_21"
df2.loc[(df2["Date"].str.contains("2016")) & ((df2["t_loc"] == "Los Cabos")), "tour_wk"] = "2016_22"
df2.loc[(df2["Date"].str.contains("2016")) & ((df2["t_loc"] == "Cincinnati")), "tour_wk"] = "2016_23"
df2.loc[(df2["Date"].str.contains("2016")) & ((df2["t_loc"] == "Winston-Salem")), "tour_wk"] = "2016_24"
df2.loc[(df2["Date"].str.contains("2016")) & ((df2["t_loc"] == "Flushing Meadows")), "tour_wk"] = "2016_25"
df2.loc[(df2["Date"].str.contains("2016")) & ((df2["t_loc"] == "Metz")|(df2["t_loc"] == "St. Petersburg")), "tour_wk"] = "2016_26"  
df2.loc[(df2["Date"].str.contains("2016")) & ((df2["t_loc"] == "Chengdu")|(df2["t_loc"] == "Shenzhen")), "tour_wk"] = "2016_27"  
df2.loc[(df2["Date"].str.contains("2016")) & ((df2["t_loc"] == "Beijing")|(df2["t_loc"] == "Tokyo")), "tour_wk"] = "2016_28"  
df2.loc[(df2["Date"].str.contains("2016")) & ((df2["t_loc"] == "Shanghai")), "tour_wk"] = "2016_29"
df2.loc[(df2["Date"].str.contains("2016")) & (((df2["t_loc"] == "Antwerp")|(df2["t_loc"] == "Moscow")|(df2["t_loc"] == "Stockholm"))), "tour_wk"] = "2016_30"
df2.loc[(df2["Date"].str.contains("2016")) & ((df2["t_loc"] == "Basel")|(df2["t_loc"] == "Vienna")), "tour_wk"] = "2016_31"
df2.loc[(df2["Date"].str.contains("2016")) & ((df2["t_loc"] == "Paris")), "tour_wk"] = "2016_32"
df2.loc[(df2["Date"].str.contains("2016")) & ((df2["t_loc"] == "London")), "tour_wk"] = "2016_33"

df2.loc[(df2["Date"].str.contains("2015")) & (((df2["t_loc"] == "Brisbane")|(df2["t_loc"] == "Doha")|(df2["t_loc"] == "Chennai"))), "tour_wk"] = "2015_01"
df2.loc[(df2["Date"].str.contains("2015")) & ((df2["t_loc"] == "Auckland")|(df2["t_loc"] == "Sydney")), "tour_wk"] = "2015_02"       
df2.loc[(df2["Date"].str.contains("2015")) & ((df2["t_loc"] == "Melbourne")), "tour_wk"] = "2015_03"       
df2.loc[(df2["Date"].str.contains("2015")) & ((df2["t_loc"] == "Quito")|(df2["t_loc"] == "Montpellier")|(df2["t_loc"] == "Zagreb")), "tour_wk"] = "2015_04"     
df2.loc[(df2["Date"].str.contains("2015")) & ((df2["t_loc"] == "Sao Paulo")|(df2["t_loc"] == "Rotterdam")|(df2["t_loc"] == "Memphis")), "tour_wk"] = "2015_05" 
df2.loc[(df2["Date"].str.contains("2015")) & ((df2["t_loc"] == "Delray Beach")|(df2["t_loc"] == "Marseille")|(df2["t_loc"] == "Rio de Janeiro")), "tour_wk"] = "2015_06"
df2.loc[(df2["Date"].str.contains("2015")) & ((df2["t_loc"] == "Acapulco")|(df2["t_loc"] == "Dubai")|(df2["t_loc"] == "Buenos Aires")), "tour_wk"] = "2015_07"
df2.loc[(df2["Date"].str.contains("2015")) & ((df2["t_loc"] == "Indian Wells")), "tour_wk"] = "2015_08"  
df2.loc[(df2["Date"].str.contains("2015")) & ((df2["t_loc"] == "Miami")), "tour_wk"] = "2015_09"
df2.loc[(df2["Date"].str.contains("2015")) & ((df2["t_loc"] == "Houston")|(df2["t_loc"] == "Casablanca")), "tour_wk"] = "2015_10"  
df2.loc[(df2["Date"].str.contains("2015")) & ((df2["t_loc"] == "Monte Carlo")), "tour_wk"] = "2015_11"
df2.loc[(df2["Date"].str.contains("2015")) & ((df2["t_loc"] == "Barcelona")|(df2["t_loc"] == "Bucharest")), "tour_wk"] = "2015_12"  
df2.loc[(df2["Date"].str.contains("2015")) & ((df2["t_loc"] == "Estoril")|(df2["t_loc"] == "Munich") |(df2["t_loc"] == "Istanbul")), "tour_wk"] = "2015_13"
df2.loc[(df2["Date"].str.contains("2015")) & ((df2["t_loc"] == "Madrid")), "tour_wk"] = "2015_14"
df2.loc[(df2["Date"].str.contains("2015")) & ((df2["t_loc"] == "Rome")), "tour_wk"] = "2015_15"
df2.loc[(df2["Date"].str.contains("2015")) & ((df2["t_loc"] == "Geneva")|(df2["t_loc"] == "Nice")), "tour_wk"] = "2015_16"
df2.loc[(df2["Date"].str.contains("2015")) & ((df2["t_loc"] == "Roland Garros")), "tour_wk"] = "2015_17"
df2.loc[(df2["Date"].str.contains("2015")) & ((df2["t_loc"] == "Bastad") |(df2["t_loc"] == "Umag")|(df2["t_loc"] == "Bogota")), "tour_wk"] = "2015_18"
df2.loc[(df2["Date"].str.contains("2015")) & ((df2["t_loc"] == "Hamburg")|(df2["t_loc"] == "Gstaad")|(df2["t_loc"] == "Atlanta")), "tour_wk"] = "2015_19"
df2.loc[(df2["Date"].str.contains("2015")) & ((df2["t_loc"] == "Kitzbuhel")|(df2["t_loc"] == "Washington")), "tour_wk"] = "2015_20"                                            
df2.loc[(df2["Date"].str.contains("2015")) & ((df2["t_loc"] == "Montreal")), "tour_wk"] = "2015_21"
df2.loc[(df2["Date"].str.contains("2015")) & ((df2["t_loc"] == "Cincinnati")), "tour_wk"] = "2015_22"
df2.loc[(df2["Date"].str.contains("2015")) & ((df2["t_loc"] == "Winston-Salem")), "tour_wk"] = "2015_23"
df2.loc[(df2["Date"].str.contains("2015")) & ((df2["t_loc"] == "Flushing Meadows")), "tour_wk"] = "2015_24"
df2.loc[(df2["Date"].str.contains("2015")) & ((df2["t_loc"] == "Metz")|(df2["t_loc"] == "St. Petersburg")), "tour_wk"] = "2015_25"  
df2.loc[(df2["Date"].str.contains("2015")) & ((df2["t_loc"] == "Kuala Lumpur")|(df2["t_loc"] == "Shenzhen")), "tour_wk"] = "2015_26"  
df2.loc[(df2["Date"].str.contains("2015")) & ((df2["t_loc"] == "Beijing")|(df2["t_loc"] == "Tokyo")), "tour_wk"] = "2015_27"  
df2.loc[(df2["Date"].str.contains("2015")) & ((df2["t_loc"] == "Shanghai")), "tour_wk"] = "2015_28"
df2.loc[(df2["Date"].str.contains("2015")) & (((df2["t_loc"] == "Vienna")|(df2["t_loc"] == "Moscow")|(df2["t_loc"] == "Stockholm"))), "tour_wk"] = "2015_29"
df2.loc[(df2["Date"].str.contains("2015")) & ((df2["t_loc"] == "Basel")|(df2["t_loc"] == "Valencia")), "tour_wk"] = "2015_30"
df2.loc[(df2["Date"].str.contains("2015")) & ((df2["t_loc"] == "Paris")), "tour_wk"] = "2015_31"
df2.loc[(df2["Date"].str.contains("2015")) & ((df2["t_loc"] == "London")), "tour_wk"] = "2015_32"                                           

df2.loc[(df2["Date"].str.contains("2014")) & (((df2["t_loc"] == "Brisbane")|(df2["t_loc"] == "Doha")|(df2["t_loc"] == "Chennai"))), "tour_wk"] = "2014_01"
df2.loc[(df2["Date"].str.contains("2014")) & ((df2["t_loc"] == "Auckland")|(df2["t_loc"] == "Sydney")), "tour_wk"] = "2014_02"       
df2.loc[(df2["Date"].str.contains("2014")) & ((df2["t_loc"] == "Melbourne")), "tour_wk"] = "2014_03"       
df2.loc[(df2["Date"].str.contains("2014")) & ((df2["t_loc"] == "Vina del Mar")|(df2["t_loc"] == "Montpellier")|(df2["t_loc"] == "Zagreb")), "tour_wk"] = "2014_04"     
df2.loc[(df2["Date"].str.contains("2014")) & ((df2["t_loc"] == "Buenos Aires")|(df2["t_loc"] == "Rotterdam")|(df2["t_loc"] == "Memphis")), "tour_wk"] = "2014_05" 
df2.loc[(df2["Date"].str.contains("2014")) & ((df2["t_loc"] == "Delray Beach")|(df2["t_loc"] == "Marseille")|(df2["t_loc"] == "Rio de Janeiro")), "tour_wk"] = "2014_06"
df2.loc[(df2["Date"].str.contains("2014")) & ((df2["t_loc"] == "Acapulco")|(df2["t_loc"] == "Dubai")|(df2["t_loc"] == "Sao Paulo")), "tour_wk"] = "2014_07"
df2.loc[(df2["Date"].str.contains("2014")) & ((df2["t_loc"] == "Indian Wells")), "tour_wk"] = "2014_08"  
df2.loc[(df2["Date"].str.contains("2014")) & ((df2["t_loc"] == "Miami")), "tour_wk"] = "2014_09"
df2.loc[(df2["Date"].str.contains("2014")) & ((df2["t_loc"] == "Houston")|(df2["t_loc"] == "Casablanca")), "tour_wk"] = "2014_10"  
df2.loc[(df2["Date"].str.contains("2014")) & ((df2["t_loc"] == "Monte Carlo")), "tour_wk"] = "2014_11"
df2.loc[(df2["Date"].str.contains("2014")) & ((df2["t_loc"] == "Barcelona")|(df2["t_loc"] == "Bucharest")), "tour_wk"] = "2014_12"  
df2.loc[(df2["Date"].str.contains("2014")) & ((df2["t_loc"] == "Oeiras")|(df2["t_loc"] == "Munich")), "tour_wk"] = "2014_13"
df2.loc[(df2["Date"].str.contains("2014")) & ((df2["t_loc"] == "Madrid")), "tour_wk"] = "2014_14"
df2.loc[(df2["Date"].str.contains("2014")) & ((df2["t_loc"] == "Rome")), "tour_wk"] = "2014_15"
df2.loc[(df2["Date"].str.contains("2014")) & ((df2["t_loc"] == "Dusseldorf")|(df2["t_loc"] == "Nice")), "tour_wk"] = "2014_16"
df2.loc[(df2["Date"].str.contains("2014")) & ((df2["t_loc"] == "Roland Garros")), "tour_wk"] = "2014_17"
df2.loc[(df2["Date"].str.contains("2014")) & ((df2["t_loc"] == "Bastad") |(df2["t_loc"] == "Stuttgart")), "tour_wk"] = "2014_18"
df2.loc[(df2["Date"].str.contains("2014")) & ((df2["t_loc"] == "Hamburg")|(df2["t_loc"] == "Bogota")), "tour_wk"] = "2014_19"
df2.loc[(df2["Date"].str.contains("2014")) & ((df2["t_loc"] == "Atlanta")|(df2["t_loc"] == "Gstaad") |(df2["t_loc"] == "Umag")), "tour_wk"] = "2014_20"
df2.loc[(df2["Date"].str.contains("2014")) & ((df2["t_loc"] == "Kitzbuhel")|(df2["t_loc"] == "Washington")), "tour_wk"] = "2014_21"                                          
df2.loc[(df2["Date"].str.contains("2014")) & ((df2["t_loc"] == "Toronto")), "tour_wk"] = "2014_22"
df2.loc[(df2["Date"].str.contains("2014")) & ((df2["t_loc"] == "Cincinnati")), "tour_wk"] = "2014_23"
df2.loc[(df2["Date"].str.contains("2014")) & ((df2["t_loc"] == "Winston-Salem")), "tour_wk"] = "2014_24"
df2.loc[(df2["Date"].str.contains("2014")) & ((df2["t_loc"] == "Flushing Meadows")), "tour_wk"] = "2014_25"
df2.loc[(df2["Date"].str.contains("2014")) & ((df2["t_loc"] == "Metz")), "tour_wk"] = "2014_26"  
df2.loc[(df2["Date"].str.contains("2014")) & ((df2["t_loc"] == "Kuala Lumpur")|(df2["t_loc"] == "Shenzhen")), "tour_wk"] = "2014_27"  
df2.loc[(df2["Date"].str.contains("2014")) & ((df2["t_loc"] == "Beijing")|(df2["t_loc"] == "Tokyo")), "tour_wk"] = "2014_28"  
df2.loc[(df2["Date"].str.contains("2014")) & ((df2["t_loc"] == "Shanghai")), "tour_wk"] = "2014_29"
df2.loc[(df2["Date"].str.contains("2014")) & (((df2["t_loc"] == "Vienna")|(df2["t_loc"] == "Moscow")|(df2["t_loc"] == "Stockholm"))), "tour_wk"] = "2014_30"
df2.loc[(df2["Date"].str.contains("2014")) & ((df2["t_loc"] == "Basel")|(df2["t_loc"] == "Valencia")), "tour_wk"] = "2014_31"
df2.loc[(df2["Date"].str.contains("2014")) & ((df2["t_loc"] == "Paris")), "tour_wk"] = "2014_32"
df2.loc[(df2["Date"].str.contains("2014")) & ((df2["t_loc"] == "London")), "tour_wk"] = "2014_33"  

df2.loc[(df2["Date"].str.contains("2013")) & (((df2["t_loc"] == "Brisbane")|(df2["t_loc"] == "Doha")|(df2["t_loc"] == "Chennai"))), "tour_wk"] = "2013_01"
df2.loc[(df2["Date"].str.contains("2013")) & ((df2["t_loc"] == "Auckland")|(df2["t_loc"] == "Sydney")), "tour_wk"] = "2013_02"       
df2.loc[(df2["Date"].str.contains("2013")) & ((df2["t_loc"] == "Melbourne")), "tour_wk"] = "2013_03"       
df2.loc[(df2["Date"].str.contains("2013")) & ((df2["t_loc"] == "Vina del Mar")|(df2["t_loc"] == "Montpellier")|(df2["t_loc"] == "Zagreb")), "tour_wk"] = "2013_04"     
df2.loc[(df2["Date"].str.contains("2013")) & ((df2["t_loc"] == "San Jose")|(df2["t_loc"] == "Rotterdam")|(df2["t_loc"] == "Sao Paulo")), "tour_wk"] = "2013_05" 
df2.loc[(df2["Date"].str.contains("2013")) & ((df2["t_loc"] == "Buenos Aires")|(df2["t_loc"] == "Marseille")|(df2["t_loc"] == "Memphis")), "tour_wk"] = "2013_06"
df2.loc[(df2["Date"].str.contains("2013")) & ((df2["t_loc"] == "Acapulco")|(df2["t_loc"] == "Dubai")|(df2["t_loc"] == "Delray Beach")), "tour_wk"] = "2013_07"
df2.loc[(df2["Date"].str.contains("2013")) & ((df2["t_loc"] == "Indian Wells")), "tour_wk"] = "2013_08"  
df2.loc[(df2["Date"].str.contains("2013")) & ((df2["t_loc"] == "Miami")), "tour_wk"] = "2013_09"
df2.loc[(df2["Date"].str.contains("2013")) & ((df2["t_loc"] == "Houston")|(df2["t_loc"] == "Casablanca")), "tour_wk"] = "2013_10"  
df2.loc[(df2["Date"].str.contains("2013")) & ((df2["t_loc"] == "Monte Carlo")), "tour_wk"] = "2013_11"
df2.loc[(df2["Date"].str.contains("2013")) & ((df2["t_loc"] == "Barcelona")|(df2["t_loc"] == "Bucharest")), "tour_wk"] = "2013_12"  
df2.loc[(df2["Date"].str.contains("2013")) & ((df2["t_loc"] == "Oeiras")|(df2["t_loc"] == "Munich")), "tour_wk"] = "2013_13"
df2.loc[(df2["Date"].str.contains("2013")) & ((df2["t_loc"] == "Madrid")), "tour_wk"] = "2013_14"
df2.loc[(df2["Date"].str.contains("2013")) & ((df2["t_loc"] == "Rome")), "tour_wk"] = "2013_15"
df2.loc[(df2["Date"].str.contains("2013")) & ((df2["t_loc"] == "Dusseldorf")|(df2["t_loc"] == "Nice")), "tour_wk"] = "2013_16"
df2.loc[(df2["Date"].str.contains("2013")) & ((df2["t_loc"] == "Roland Garros")), "tour_wk"] = "2013_17"
df2.loc[(df2["Date"].str.contains("2013")) & ((df2["t_loc"] == "Bastad") |(df2["t_loc"] == "Stuttgart")), "tour_wk"] = "2013_18"
df2.loc[(df2["Date"].str.contains("2013")) & ((df2["t_loc"] == "Hamburg")|(df2["t_loc"] == "Bogota")), "tour_wk"] = "2013_19"
df2.loc[(df2["Date"].str.contains("2013")) & ((df2["t_loc"] == "Atlanta")|(df2["t_loc"] == "Gstaad") |(df2["t_loc"] == "Umag")), "tour_wk"] = "2013_20"
df2.loc[(df2["Date"].str.contains("2013")) & ((df2["t_loc"] == "Kitzbuhel")|(df2["t_loc"] == "Washington")), "tour_wk"] = "2013_21"                                          
df2.loc[(df2["Date"].str.contains("2013")) & ((df2["t_loc"] == "Montreal")), "tour_wk"] = "2013_22"
df2.loc[(df2["Date"].str.contains("2013")) & ((df2["t_loc"] == "Cincinnati")), "tour_wk"] = "2013_23"
df2.loc[(df2["Date"].str.contains("2013")) & ((df2["t_loc"] == "Winston-Salem")), "tour_wk"] = "2013_24"
df2.loc[(df2["Date"].str.contains("2013")) & ((df2["t_loc"] == "Flushing Meadows")), "tour_wk"] = "2013_25"
df2.loc[(df2["Date"].str.contains("2013")) & ((df2["t_loc"] == "Metz")|(df2["t_loc"] == "St. Petersburg")), "tour_wk"] = "2013_26"  
df2.loc[(df2["Date"].str.contains("2013")) & ((df2["t_loc"] == "Kuala Lumpur")|(df2["t_loc"] == "Bangkok")), "tour_wk"] = "2013_27"  
df2.loc[(df2["Date"].str.contains("2013")) & ((df2["t_loc"] == "Beijing")|(df2["t_loc"] == "Tokyo")), "tour_wk"] = "2013_28"  
df2.loc[(df2["Date"].str.contains("2013")) & ((df2["t_loc"] == "Shanghai")), "tour_wk"] = "2013_29"
df2.loc[(df2["Date"].str.contains("2013")) & (((df2["t_loc"] == "Vienna")|(df2["t_loc"] == "Moscow")|(df2["t_loc"] == "Stockholm"))), "tour_wk"] = "2013_30"
df2.loc[(df2["Date"].str.contains("2013")) & ((df2["t_loc"] == "Basel")|(df2["t_loc"] == "Valencia")), "tour_wk"] = "2013_31"
df2.loc[(df2["Date"].str.contains("2013")) & ((df2["t_loc"] == "Paris")), "tour_wk"] = "2013_32"
df2.loc[(df2["Date"].str.contains("2013")) & ((df2["t_loc"] == "London")), "tour_wk"] = "2013_33"  

df2.loc[(df2["Date"].str.contains("2012")) & (((df2["t_loc"] == "Brisbane")|(df2["t_loc"] == "Doha")|(df2["t_loc"] == "Chennai"))), "tour_wk"] = "2012_01"
df2.loc[(df2["Date"].str.contains("2012")) & ((df2["t_loc"] == "Auckland")|(df2["t_loc"] == "Sydney")), "tour_wk"] = "2012_02"       
df2.loc[(df2["Date"].str.contains("2012")) & ((df2["t_loc"] == "Melbourne")), "tour_wk"] = "2012_03"       
df2.loc[(df2["Date"].str.contains("2012")) & ((df2["t_loc"] == "Vina del Mar")|(df2["t_loc"] == "Montpellier")|(df2["t_loc"] == "Zagreb")), "tour_wk"] = "2012_04"     
df2.loc[(df2["Date"].str.contains("2012")) & ((df2["t_loc"] == "San Jose")|(df2["t_loc"] == "Rotterdam")|(df2["t_loc"] == "Sao Paulo")), "tour_wk"] = "2012_05" 
df2.loc[(df2["Date"].str.contains("2012")) & ((df2["t_loc"] == "Buenos Aires")|(df2["t_loc"] == "Marseille")|(df2["t_loc"] == "Memphis")), "tour_wk"] = "2012_06"
df2.loc[(df2["Date"].str.contains("2012")) & ((df2["t_loc"] == "Acapulco")|(df2["t_loc"] == "Dubai")|(df2["t_loc"] == "Delray Beach")), "tour_wk"] = "2012_07"
df2.loc[(df2["Date"].str.contains("2012")) & ((df2["t_loc"] == "Indian Wells")), "tour_wk"] = "2012_08"  
df2.loc[(df2["Date"].str.contains("2012")) & ((df2["t_loc"] == "Miami")), "tour_wk"] = "2012_09"
df2.loc[(df2["Date"].str.contains("2012")) & ((df2["t_loc"] == "Houston")|(df2["t_loc"] == "Casablanca")), "tour_wk"] = "2012_10"  
df2.loc[(df2["Date"].str.contains("2012")) & ((df2["t_loc"] == "Monte Carlo")), "tour_wk"] = "2012_11"
df2.loc[(df2["Date"].str.contains("2012")) & ((df2["t_loc"] == "Barcelona")|(df2["t_loc"] == "Bucharest")), "tour_wk"] = "2012_12"  
df2.loc[(df2["Date"].str.contains("2012")) & ((df2["t_loc"] == "Estoril")|(df2["t_loc"] == "Munich") |(df2["t_loc"] == "Belgrade")), "tour_wk"] = "2012_13"
df2.loc[(df2["Date"].str.contains("2012")) & ((df2["t_loc"] == "Madrid")), "tour_wk"] = "2012_14"
df2.loc[(df2["Date"].str.contains("2012")) & ((df2["t_loc"] == "Rome")), "tour_wk"] = "2012_15"
df2.loc[(df2["Date"].str.contains("2012")) & ((df2["t_loc"] == "Dusseldorf")|(df2["t_loc"] == "Nice")), "tour_wk"] = "2012_16"
df2.loc[(df2["Date"].str.contains("2012")) & ((df2["t_loc"] == "Roland Garros")), "tour_wk"] = "2012_17"
df2.loc[(df2["Date"].str.contains("2012")) & ((df2["t_loc"] == "Bastad") |(df2["t_loc"] == "Stuttgart") |(df2["t_loc"] == "Umag")), "tour_wk"] = "2012_18"
df2.loc[(df2["Date"].str.contains("2012")) & ((df2["t_loc"] == "Atlanta")|(df2["t_loc"] == "Gstaad") |(df2["t_loc"] == "Hamburg")), "tour_wk"] = "2012_19"
df2.loc[(df2["Date"].str.contains("2012")) & ((df2["t_loc"] == "Kitzbuhel")|(df2["t_loc"] == "Los Angeles")), "tour_wk"] = "2012_20"                                          
df2.loc[(df2["Date"].str.contains("2012")) & ((df2["t_loc"] == "Washington")), "tour_wk"] = "2012_21"
df2.loc[(df2["Date"].str.contains("2012")) & ((df2["t_loc"] == "Toronto")), "tour_wk"] = "2012_22"
df2.loc[(df2["Date"].str.contains("2012")) & ((df2["t_loc"] == "Cincinnati")), "tour_wk"] = "2012_23"
df2.loc[(df2["Date"].str.contains("2012")) & ((df2["t_loc"] == "Winston-Salem")), "tour_wk"] = "2012_24"
df2.loc[(df2["Date"].str.contains("2012")) & ((df2["t_loc"] == "Flushing Meadows")), "tour_wk"] = "2012_25"
df2.loc[(df2["Date"].str.contains("2012")) & ((df2["t_loc"] == "Metz")|(df2["t_loc"] == "St. Petersburg")), "tour_wk"] = "2012_26"  
df2.loc[(df2["Date"].str.contains("2012")) & ((df2["t_loc"] == "Kuala Lumpur")|(df2["t_loc"] == "Bangkok")), "tour_wk"] = "2012_27"  
df2.loc[(df2["Date"].str.contains("2012")) & ((df2["t_loc"] == "Beijing")|(df2["t_loc"] == "Tokyo")), "tour_wk"] = "2012_28"  
df2.loc[(df2["Date"].str.contains("2012")) & ((df2["t_loc"] == "Shanghai")), "tour_wk"] = "2012_29"
df2.loc[(df2["Date"].str.contains("2012")) & (((df2["t_loc"] == "Vienna")|(df2["t_loc"] == "Moscow")|(df2["t_loc"] == "Stockholm"))), "tour_wk"] = "2012_30"
df2.loc[(df2["Date"].str.contains("2012")) & ((df2["t_loc"] == "Basel")|(df2["t_loc"] == "Valencia")), "tour_wk"] = "2012_31"
df2.loc[(df2["Date"].str.contains("2012")) & ((df2["t_loc"] == "Paris")), "tour_wk"] = "2012_32"
df2.loc[(df2["Date"].str.contains("2012")) & ((df2["t_loc"] == "London")), "tour_wk"] = "2012_33"  


We will now calculate the implied win odds from the average wagering lines per match. This requires several data transformations and the removal of the vig. 

In [12]:
# Convert decimal odds to American odds
df2["AvgW_AO"] = ""
df2["AvgL_AO"] = ""

df2.loc[(df2["AvgW_BL"] >= 2), "AvgW_AO"] = (df2["AvgW_BL"] - 1) * 100
df2.loc[(df2["AvgW_BL"] < 2), "AvgW_AO"] = (-100)/(df2["AvgW_BL"] - 1) 

df2.loc[(df2["AvgL_BL"] >= 2), "AvgL_AO"] = (df2["AvgL_BL"] - 1) * 100
df2.loc[(df2["AvgL_BL"] < 2), "AvgL_AO"] = (-100)/(df2["AvgL_BL"] - 1)

df2["AvgW_AO"] = pd.to_numeric(df2["AvgW_AO"])
df2["AvgL_AO"] = pd.to_numeric(df2["AvgL_AO"])

In [13]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18353 entries, 0 to 20973
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   t_loc    18353 non-null  object 
 1   t_name   18353 non-null  object 
 2   Date     18353 non-null  object 
 3   t_surf   18353 non-null  object 
 4   t_round  18353 non-null  object 
 5   w_name   18353 non-null  object 
 6   l_name   18353 non-null  object 
 7   w_rank   18344 non-null  float64
 8   l_rank   18307 non-null  float64
 9   AvgW_BL  18339 non-null  float64
 10  AvgL_BL  18339 non-null  float64
 11  Comment  18353 non-null  object 
 12  tour_wk  18353 non-null  object 
 13  AvgW_AO  18339 non-null  float64
 14  AvgL_AO  18339 non-null  float64
dtypes: float64(6), object(9)
memory usage: 2.9+ MB


In [14]:
# Convert American Odds data to implied odds
df2["AvgW_IP"] = ""
df2["AvgL_IP"] = ""

df2.loc[(df2["AvgW_AO"] < 0), "AvgW_IP"] = abs(df2["AvgW_AO"])/(abs(df2["AvgW_AO"]) + 100) *100  #favorite winners
df2.loc[(df2["AvgW_AO"] > 0), "AvgW_IP"] = 100/(abs(df2["AvgW_AO"]) + 100) *100  #underdog winners

df2.loc[(df2["AvgL_AO"] < 0), "AvgL_IP"] = abs(df2["AvgL_AO"])/(abs(df2["AvgL_AO"]) + 100) *100  #favorite losers
df2.loc[(df2["AvgL_AO"] > 0), "AvgL_IP"] = 100/(abs(df2["AvgL_AO"]) + 100) *100  #underdog losers

df2["AvgW_IP"] = pd.to_numeric(df2["AvgW_IP"])
df2["AvgL_IP"] = pd.to_numeric(df2["AvgL_IP"])

In [18]:
# Remove the vig from the implied probabilities
df2["AvgW_IP_NV"] = ""
df2["AvgL_IP_NV"] = ""

df2["AvgW_IP_NV"] = df2["AvgW_IP"]/(df2["AvgW_IP"] + df2["AvgL_IP"]) * 100
df2["AvgL_IP_NV"] = df2["AvgL_IP"]/(df2["AvgW_IP"] + df2["AvgL_IP"]) * 100

#df2["AvgW_IP_NV"] = pd.to_numeric(df2["AvgW_IP_NV"])
#df2["AvgL_IP_NV"] = pd.to_numeric(df2["AvgL_IP_NV"])

df2["AvgW_IP_NV"] = round(pd.to_numeric(df2["AvgW_IP_NV"]), 2)
df2["AvgL_IP_NV"] = round(pd.to_numeric(df2["AvgL_IP_NV"]), 2)

Now we have the vig-removed implied probabilities that we want for modeling.

In [19]:
# Save hard court historical wagering data 
df2_hard = df2[df2['t_surf'].str.contains("Hard")]
df2_hard.to_csv('../data/merged_wagering_hard.csv', index=False)

In [20]:
# Save clay court historical wagering data 
df2_clay = df2[df2['t_surf'].str.contains("Clay")]
df2_clay.to_csv('../data/merged_wagering_clay.csv', index=False)

# Note that even though in the Tennis Prediction Project I arrived at a more expansive date range for inclusion for clay courts than hard 
# (2009-2019) the first 3 years in that range are just for stats accrual, so we don't need wagering data from 2009-2012. 